In [1]:
# Matplotlib config
%matplotlib inline
%config InlineBackend.figure_formats = ['svg']
%config InlineBackend.rc = {'figure.figsize': (5.0, 3.0)}

import numpy as np
import pandas as pd
import seaborn as sns

import gzip
import numpy as np

# Code aufbauend auf: https://stackoverflow.com/a/62781370
def load_images(path):
    with gzip.open(path, 'r') as f:
        # first 4 bytes is a magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        # second 4 bytes is the number of images
        image_count = int.from_bytes(f.read(4), 'big')
        # third 4 bytes is the row count
        row_count = int.from_bytes(f.read(4), 'big')
        # fourth 4 bytes is the column count
        column_count = int.from_bytes(f.read(4), 'big')
        # rest is the image pixel data, each pixel is stored as an unsigned byte
        # pixel values are 0 to 255
        image_data = f.read()
        images = np.frombuffer(image_data, dtype=np.uint8)\
            .reshape((image_count, row_count, column_count))
        return images
    
def load_labels(path):
    with gzip.open(path, 'r') as f:
        # first 4 bytes is a magic number
        magic_number = int.from_bytes(f.read(4), 'big')
        # second 4 bytes is the number of labels
        label_count = int.from_bytes(f.read(4), 'big')
        # rest is the label data, each label is stored as unsigned byte
        # label values are 0 to 9
        label_data = f.read()
        labels = np.frombuffer(label_data, dtype=np.uint8)
        return labels

X_train = load_images("../data/MNIST/train-images-idx3-ubyte.gz")
y_train = load_labels("../data/MNIST/train-labels-idx1-ubyte.gz")

X_test = load_images("../data/MNIST/t10k-images-idx3-ubyte.gz")
y_test = load_labels("../data/MNIST/t10k-labels-idx1-ubyte.gz")

X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

X_train = X_train.astype(np.float32) / 255
X_test = X_test.astype(np.float32) / 255

from tensorflow.keras.utils import to_categorical

y_train = to_categorical(y_train)

In [2]:
# Tensorflow laden
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    keras.Input(shape = (28, 28, 1)),
    layers.Conv2D(10, kernel_size = (3, 3), padding = "same", activation = "relu"),
    layers.MaxPooling2D(),
    layers.Conv2D(10, kernel_size = (3, 3), padding = "same", activation = "relu"),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(100, activation = "relu"),
    layers.Dense(10, activation = "softmax")
])

print(model.summary())

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 28, 28, 10)     │           100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 14, 14, 10)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 14, 14, 10)     │           910 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 7, 7, 10)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 490)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 100)            │        49,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,010 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 51,120 (199.69 KB)

 Trainable params: 51,120 (199.69 KB)

 Non-trainable params: 0 (0.00 B)

None


In [3]:
model.compile(
    optimizer = keras.optimizers.RMSprop(),
    loss = keras.losses.CategoricalCrossentropy(),
    metrics = [
        keras.metrics.CategoricalAccuracy()
    ]
)

model.fit(X_train, y_train, 
          batch_size = 64, 
          epochs = 20, 
          validation_split = 0.1)

Epoch 1/20
844/844 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - categorical_accuracy: 0.8346 - loss: 0.5602 - val_categorical_accuracy: 0.9768 - val_loss: 0.0764
Epoch 2/20
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - categorical_accuracy: 0.9748 - loss: 0.0828 - val_categorical_accuracy: 0.9815 - val_loss: 0.0572
Epoch 3/20
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - categorical_accuracy: 0.9823 - loss: 0.0577 - val_categorical_accuracy: 0.9823 - val_loss: 0.0671
Epoch 4/20
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - categorical_accuracy: 0.9864 - loss: 0.0432 - val_categorical_accuracy: 0.9868 - val_loss: 0.0418
Epoch 5/20
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - categorical_accuracy: 0.9890 - loss: 0.0330 - val_categorical_accuracy: 0.9902 - val_loss: 0.0391
Epoch 6/20
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - categorical_accuracy: 0.9907 - loss: 0.0308 - val_categorical_accuracy: 0.9887 - val_loss: 0.0441
Epoch 7/20
844/844 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - categorical_accuracy: 0.9920 - los

In [7]:
model.save("my-model.tf.keras")